In [1]:
import urllib
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from os.path import isfile

import torch
from torchvision import transforms
import torchvision.transforms.functional as TF

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torch.nn.functional as F
from torch import nn

import albumentations as A
from albumentations.pytorch import transforms

import numpy as np
from PIL import Image
import imageio
from matplotlib import pyplot
%matplotlib inline

In [2]:
#전역변수 설정

url = "https://www.cs.toronto.edu/~vmnih/data/"

train_input_url = "mass_roads/train/sat/index.html"
train_target_url = "mass_roads/train/map/index.html"

train_directory = "C:/Users/spinsi/Desktop/code/jupyternotebook_python37/2. Road extraction/datasets/train/input"
target_directory = "C:/Users/spinsi/Desktop/code/jupyternotebook_python37/2. Road extraction/datasets/train/target"

dir_train = 'C:/Users/spinsi/Desktop/code/jupyternotebook_python37/2. Road extraction/datasets/train/input_filtered'
dir_target = 'C:/Users/spinsi/Desktop/code/jupyternotebook_python37/2. Road extraction/datasets/train/target_filtered'

In [5]:
#DataSet 설정 및 Trasform 과정
#train 파일(R,G,B)을 img1에 저장 mask(binary)를 img2에 저장 함
#두 파일을 합쳐서 4차원으로 stacking 하고 img3로 지정, normalization 및 crop 진행
#crop 진행 후, img3를 다시 분할해서 img1과 img2로 나눔

class CustomDataset(Dataset): 
    def __init__(self, train_paths, target_paths, transform=None): 
        self.train_paths = [f for f in os.listdir(dir_train) if isfile(dir_train + '/'+ f)] 
        self.target_paths = [f for f in os.listdir(dir_target) if isfile(dir_target + '/' + f)] 
        self.transform = transform
        
    def __getitem__(self, index): 
        train = np.array(Image.open(dir_train+'/'+self.train_paths[index]))
        target = np.array(Image.open(dir_target+'/'+self.target_paths[index]))
        
        if self.transform:
            #transforms.Compose
            data = self.transform(image=train, mask=target)
            data_img = data["image"]
            data_lab = data["mask"]
            
            data = {'train': data_img, 'target': data_lab}
        return data
        
    def __len__(self):
        return len(self.train_paths)
    
trans = A.Compose([
    A.HorizontalFlip(),
    A.VerticalFlip(),
    A.Normalize(mean=0.5, std=0.5),
    transforms.ToTensorV2(transpose_mask=True)
])



In [6]:
#Unet 모델 쌓기

class UNet(nn.Module):  
    def __init__(self): 
# super(subclass, self) : subclass에서 base class의 내용을 오버라이드해서 사용하고 싶을 때
        super(UNet, self).__init__() 
        
# 네트워크에서 반복적으로 사용되는 Conv + BatchNorm + Relu를 합쳐서 하나의 함수로 정의
       	def CBR2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True):
            layers = []
            layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                 kernel_size=kernel_size, stride=stride, padding=padding,
                                 bias=bias)]
            layers += [nn.BatchNorm2d(num_features=out_channels)]
            layers += [nn.ReLU()]

            cbr = nn.Sequential(*layers) # *으로 list unpacking 

            return cbr

        # Contracting path
        self.enc1_1 = CBR2d(in_channels=3, out_channels=64)
        self.enc1_2 = CBR2d(in_channels=64, out_channels=64)
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.enc2_1 = CBR2d(in_channels=64, out_channels=128)
        self.enc2_2 = CBR2d(in_channels=128, out_channels=128)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        
        self.enc3_1 = CBR2d(in_channels=128, out_channels=256)
        self.enc3_2 = CBR2d(in_channels=256, out_channels=256)
        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.enc4_1 = CBR2d(in_channels=256, out_channels=512)
        self.enc4_2 = CBR2d(in_channels=512, out_channels=512)
        self.pool4 = nn.MaxPool2d(kernel_size=2)
        
        self.enc5_1 = CBR2d(in_channels=512, out_channels=1024)
        # Expansive path
        self.dec5_1 = CBR2d(in_channels=1024, out_channels=512)

        self.unpool4 = nn.ConvTranspose2d(in_channels=512, out_channels=512,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec4_2 = CBR2d(in_channels=2 * 512, out_channels=512)
        self.dec4_1 = CBR2d(in_channels=512, out_channels=256)
        self.unpool3 = nn.ConvTranspose2d(in_channels=256, out_channels=256,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec3_2 = CBR2d(in_channels=2 * 256, out_channels=256)
        self.dec3_1 = CBR2d(in_channels=256, out_channels=128)
        self.unpool2 = nn.ConvTranspose2d(in_channels=128, out_channels=128,
                                          kernel_size=2, stride=2, padding=0, bias=True)
        
        self.dec2_2 = CBR2d(in_channels=2 * 128, out_channels=128)
        self.dec2_1 = CBR2d(in_channels=128, out_channels=64)
        self.unpool1 = nn.ConvTranspose2d(in_channels=64, out_channels=64,
                                          kernel_size=2, stride=2, padding=0, bias=True)
        
        self.dec1_2 = CBR2d(in_channels=2 * 64, out_channels=64)
        self.dec1_1 = CBR2d(in_channels=64, out_channels=64)
        self.fc = nn.Conv2d(in_channels=64, out_channels=3, kernel_size=1)
        
# __init__ 함수에서 선언한 layer들 연결해서 data propa flow 만들기
    def forward(self, x):
        enc1_1 = self.enc1_1(x)
        enc1_2 = self.enc1_2(enc1_1)
        pool1 = self.pool1(enc1_2)
        
        enc2_1 = self.enc2_1(pool1)
        enc2_2 = self.enc2_2(enc2_1)
        pool2 = self.pool2(enc2_2)
        
        enc3_1 = self.enc3_1(pool2)
        enc3_2 = self.enc3_2(enc3_1)
        pool3 = self.pool3(enc3_2)
        
        enc4_1 = self.enc4_1(pool3)
        enc4_2 = self.enc4_2(enc4_1)
        pool4 = self.pool4(enc4_2)
        
        enc5_1 = self.enc5_1(pool4)
        dec5_1 = self.dec5_1(enc5_1)
        
        unpool4 = self.unpool4(dec5_1)
        cat4 = torch.cat((unpool4, enc4_2), dim=1)
        dec4_2 = self.dec4_2(cat4)
        dec4_1 = self.dec4_1(dec4_2)
        
        unpool3 = self.unpool3(dec4_1)
        cat3 = torch.cat((unpool3, enc3_2), dim=1)
        dec3_2 = self.dec3_2(cat3)
        dec3_1 = self.dec3_1(dec3_2)

        unpool2 = self.unpool2(dec3_1)
        cat2 = torch.cat((unpool2, enc2_2), dim=1)
        dec2_2 = self.dec2_2(cat2)
        dec2_1 = self.dec2_1(dec2_2)

        unpool1 = self.unpool1(dec2_1)
        cat1 = torch.cat((unpool1, enc1_2), dim=1)
        #tensor 합치기: Unet에서 Copy and Crop 과정을 담당한다
        dec1_2 = self.dec1_2(cat1)
        dec1_1 = self.dec1_1(dec1_2)

        out = self.fc(dec1_1)

        return out # data가 모든 layer를 거쳐서 나온 output 값

In [16]:
#datloader 설정
#train은 images1에 mask는 images2에 저장
dataset = CustomDataset(dir_train, dir_target, transform=trans)
data = dataset.__getitem__(0)

train_dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

batch_iterator = iter(train_dataloader)
images1, images2 = next(batch_iterator)



tensor([[[-0.1373,  0.1765,  0.0353,  ...,  0.2706,  0.2706, -0.2314],
         [-0.1216,  0.0039, -0.0667,  ...,  0.0980, -0.0588, -0.4667],
         [-0.0118, -0.0431,  0.0588,  ...,  0.2235,  0.1451, -0.2314],
         ...,
         [-0.5922, -0.5765, -0.6000,  ..., -0.3020, -0.3098, -0.3725],
         [-0.6078, -0.6000, -0.6078,  ..., -0.1608, -0.2314, -0.1059],
         [-0.6392, -0.6392, -0.6392,  ..., -0.1529, -0.2314,  0.0118]],

        [[-0.3176, -0.0039, -0.1137,  ...,  0.0824,  0.0824, -0.4431],
         [-0.3020, -0.1686, -0.2157,  ..., -0.0902, -0.2471, -0.6784],
         [-0.1843, -0.2157, -0.0902,  ...,  0.0353, -0.0431, -0.4431],
         ...,
         [-0.4275, -0.4353, -0.4353,  ..., -0.3647, -0.3647, -0.4275],
         [-0.4667, -0.4667, -0.4667,  ..., -0.2471, -0.3176, -0.1686],
         [-0.5059, -0.5059, -0.5059,  ..., -0.2549, -0.3176, -0.0745]],

        [[-0.4431, -0.1294, -0.2314,  ...,  0.0039, -0.0118, -0.5294],
         [-0.4275, -0.2784, -0.3333,  ..., -0

In [ ]:
#트레이닝
## 하이퍼 파라미터 설정

lr = 1e-3
batch_size = 4
num_epoch = 100

data_dir = '/content/drive/My Drive/Colab Notebooks/파이토치/Architecture practice/UNet/data'
ckpt_dir = '/content/drive/My Drive/Colab Notebooks/파이토치/Architecture practice/UNet/checkpoint'
log_dir = '/content/drive/My Drive/Colab Notebooks/파이토치/Architecture practice/UNet/log'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# transform 적용해서 데이터 셋 불러오기
dataset = CustomDataset(dir_train, dir_target, transform=trans)
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# val set도 동일하게 진행
dataset_val = Dataset(data_dir=os.path.join(data_dir,'val'),transform = transform)
loader_val = DataLoader(dataset_val, batch_size=batch_size , shuffle=True)

# 네트워크 불러오기
net = UNet().to(device) # device : cpu or gpu

# loss 정의
fn_loss = nn.BCEWithLogitsLoss().to(device)

# Optimizer 정의
optim = torch.optim.Adam(net.parameters(), lr = lr ) 

# 기타 variables 설정
num_train = len(dataset_train)
num_val = len(dataset_val)

num_train_for_epoch = np.ceil(num_train/batch_size) # np.ceil : 소수점 반올림
num_val_for_epoch = np.ceil(num_val/batch_size)

# 기타 function 설정
fn_tonumpy = lambda x : x.to('cpu').detach().numpy().transpose(0,2,3,1) # device 위에 올라간 텐서를 detach 한 뒤 numpy로 변환
fn_denorm = lambda x, mean, std : (x * std) + mean 
fn_classifier = lambda x :  1.0 * (x > 0.5)  # threshold 0.5 기준으로 indicator function으로 classifier 구현

# Tensorbord
writer_train = SummaryWriter(log_dir=os.path.join(log_dir,'train'))
writer_val = SummaryWriter(log_dir = os.path.join(log_dir,'val'))

In [ ]:
# 네트워크 저장하기
# train을 마친 네트워크 저장 
# net : 네트워크 파라미터, optim  두개를 dict 형태로 저장
def save(ckpt_dir,net,optim,epoch):
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    torch.save({'net':net.state_dict(),'optim':optim.state_dict()},'%s/model_epoch%d.pth'%(ckpt_dir,epoch))

# 네트워크 불러오기
def load(ckpt_dir,net,optim):
    if not os.path.exists(ckpt_dir): # 저장된 네트워크가 없다면 인풋을 그대로 반환
        epoch = 0
        return net, optim, epoch
    
    ckpt_lst = os.listdir(ckpt_dir) # ckpt_dir 아래 있는 모든 파일 리스트를 받아온다
    ckpt_lst.sort(key = lambda f : int(''.join(filter(str,isdigit,f))))

    dict_model = torch.load('%s/%s' % (ckpt_dir,ckpt_lst[-1]))

    net.load_state_dict(dict_model['net'])
    optim.load_state_dict(dict_model['optim'])
    epoch = int(ckpt_lst[-1].split('epoch')[1].split('.pth')[0])

    return net,optim,epoch


# 네트워크 학습시키기
start_epoch = 0
net, optim, start_epoch = load(ckpt_dir = ckpt_dir, net = net, optim = optim) # 저장된 네트워크 불러오기

for epoch in range(start_epoch+1,num_epoch +1):
    net.train()
    loss_arr = []

    for batch, data in enumerate(loader_train,1): # 1은 뭐니 > index start point
        # forward
        label = data['label'].to(device)   # 데이터 device로 올리기     
        inputs = data['input'].to(device)
        output = net(inputs) 

        # backward
        optim.zero_grad()  # gradient 초기화
        loss = fn_loss(output, label)  # output과 label 사이의 loss 계산
        loss.backward() # gradient backpropagation
        optim.step() # backpropa 된 gradient를 이용해서 각 layer의 parameters update

        # save loss
        loss_arr += [loss.item()]

        # tensorbord에 결과값들 저정하기
        label = fn_tonumpy(label)
        inputs = fn_tonumpy(fn_denorm(inputs,0.5,0.5))
        output = fn_tonumpy(fn_classifier(output))

        writer_train.add_image('label', label, num_train_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
        writer_train.add_image('input', inputs, num_train_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
        writer_train.add_image('output', output, num_train_for_epoch * (epoch - 1) + batch, dataformats='NHWC')

    writer_train.add_scalar('loss', np.mean(loss_arr), epoch)

    
    # validation
    with torch.no_grad(): # validation 이기 때문에 backpropa 진행 x, 학습된 네트워크가 정답과 얼마나 가까운지 loss만 계산
        net.eval() # 네트워크를 evaluation 용으로 선언
        loss_arr = []

        for batch, data in enumerate(loader_val,1):
            # forward
            label = data['label'].to(device)
            inputs = data['input'].to(device)
            output = net(inputs)

            # loss 
            loss = fn_loss(output,label)
            loss_arr += [loss.item()]
            print('valid : epoch %04d / %04d | Batch %04d \ %04d | Loss %04d'%(epoch,num_epoch,batch,num_val_for_epoch,np.mean(loss_arr)))

            # Tensorboard 저장하기
            label = fn_tonumpy(label)
            inputs = fn_tonumpy(fn_denorm(inputs, mean=0.5, std=0.5))
            output = fn_tonumpy(fn_classifier(output))

            writer_val.add_image('label', label, num_val_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
            writer_val.add_image('input', inputs, num_val_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
            writer_val.add_image('output', output, num_val_for_epoch * (epoch - 1) + batch, dataformats='NHWC')

        writer_val.add_scalar('loss', np.mean(loss_arr), epoch)

        # epoch이 끝날때 마다 네트워크 저장
        save(ckpt_dir=ckpt_dir, net = net, optim = optim, epoch = epoch)

writer_train.close()
writer_val.close()

In [36]:
#이미지 stacking
a = os.listdir(dir_train)
img1 = np.array(Image.open(dir_train+'/'+a[0]))
img2 = np.array(Image.open(dir_target+'/'+a[0]))

img3 = np.dstack([img1,img2])
print(img3[:,:,0:3].shape)
img3[:,:,3:4]


(1500, 1500, 3)


array([[[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       ...,

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]]], dtype=uint8)

In [8]:
#Tensor을 Image로 출력하는 함수
tf = transforms.ToPILImage()
img_t = tf(dataset.__getitem__(0)[1])
img_t.show()


AttributeError: module 'albumentations.pytorch.transforms' has no attribute 'ToPILImage'

In [35]:
#Tensor을 Image로 출력하는 함수
tf = transforms.ToPILImage()
img_t = tf(images1[0])
img_t.show()

In [38]:
print(images2[0])

tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         ...,
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.]]])


In [51]:
print(np.unique(dataset.__getitem__(0)[1]))

[-1.         -0.99215686 -0.9843137  -0.9764706  -0.96862745 -0.9607843
 -0.9529412  -0.94509804 -0.9372549  -0.92941177 -0.92156863 -0.9137255
 -0.90588236 -0.8980392  -0.8901961  -0.88235295 -0.8745098  -0.8666667
 -0.85882354 -0.8509804  -0.84313726 -0.8352941  -0.827451   -0.81960785
 -0.8117647  -0.8039216  -0.7882353  -0.78039217 -0.77254903 -0.7647059
 -0.75686276 -0.7490196  -0.7411765  -0.7254902  -0.7176471  -0.70980394
  0.7019608   0.7176471   0.7254902   0.7411765   0.7490196   0.75686276
  0.7647059   0.77254903  0.78039217  0.7882353   0.79607844  0.8039216
  0.8117647   0.81960785  0.827451    0.8352941   0.84313726  0.8509804
  0.85882354  0.8666667   0.8745098   0.88235295  0.8901961   0.8980392
  0.90588236  0.9137255   0.92156863  0.92941177  0.9372549   0.94509804
  0.9529412   0.9607843   0.96862745  0.9764706   0.9843137   0.99215686
  1.        ]
